# Annotation der Volksverhetzung im Referenzdatensatz

### 1. Als "HATE" annotierte Einträge aus dem Referenzdatensatz herausfiltern

Anmerkung:

Die Notwendigkeit, Duplikate zu entfernen, ist erst im Laufe der Annotation klar geworden.  
Daher enthielten die hier eingelesenen Daten und die aus der Annotation resultierenden Json-Dateien noch Duplikate,  
die dann nach der Annotation entfernt wurden.

Außerdem war anfangs die Überlegung den Split der Train- und Testdaten so zu belassen wie durch die Datensätze vorgegeben  
und die Daten wurden dementsprechend nacheinander annotiert;  
um später einen stratifizierten Split zu ermöglichen, wird aber ab dem Einlesen der neuen Annotation mit dem gesamten Datensatz HATE   gearbeitet;  
 der Train/Test-Split erfolgt dann in 'src/classify.ipynb'

In [ ]:
# Bereits ausgeführt, nicht noch einmal durchführen
import json

# Einlesen und filtern
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="r", encoding="utf-8") as hatetrainin:
    all_cont = hatetrainin.readlines()
    all_cont = all_cont[1:]
    sep_cont = [entry.strip().split("\t") for entry in all_cont]
    hate_train = [tweet for tweet in sep_cont if tweet[3] == "HATE"]

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="r", encoding="utf-8") as hatetestin:
    all_test = hatetestin.readlines()
    all_test = all_test[1:]
    sep_test = [entry.strip().split("\t") for entry in all_test]
    hate_test = [tweet for tweet in sep_test if tweet[3] == "HATE"]

# Als json-Dateien speichern
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE.json", mode="w", encoding="utf-8") as jsonout:
    cont_dicts = [{"id":elem[0], "text":elem[1], "tag1":elem[2], "tag2":elem[3]} for elem in hate_train]
    prep_cont = json.dumps(cont_dicts)
    jsonout.write(prep_cont)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.json", mode="w", encoding="utf-8") as jsonout_test:
    cont_dicts_test = [{"id":elem[0], "text":elem[1], "tag1":elem[2], "tag2":elem[3]} for elem in hate_test]
    prep_cont_test = json.dumps(cont_dicts_test)
    jsonout_test.write(prep_cont_test)

### 2. Annotation in doccano

### 3. Prüfung der Annotationshierarchie auf Korrektheit

In [ ]:
def check_anno(datensatz):
    """Annotationslogik prüfen und Problemfälle sammeln"""
    probleme = []
    for entry in datensatz:
        if not anno_logik_check(entry["label"]): probleme.append(entry)
    return probleme


def anno_logik_check(labelset):
    """ Die Annotationslogik überprüfen:
    Input: Liste aller Labels für einen Tweet
    Output: True (falls alles korrekt) / False (falls irgendein Problem vorliegt)    
    """
    labelset = set(labelset)
    korrekt = True
    if len(labelset) == 0:
        korrekt = False
    elif (len(labelset) == 1) and ("KEINE" not in labelset):
        korrekt = False
    else:
        # 1. VVH-ALLG interne Logik
        if "VVH-ALLG" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset == set():
                korrekt = False
            # Mind. eine Gruppe genannt
            if {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"} & labelset == set():
                korrekt = False
            # keine VVH-NS Labels
            if {"VVH-NS", "Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset != set():
                korrekt = False
            if "KEINE" in labelset: korrekt = False

        # 2. VVH-NS interne Logik
        if "VVH-NS" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset == set(): 
                korrekt = False
            # keine VVH-ALLG Labels
            if {"VVH-ALLG", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset != set():
                korrekt = False 
            if "KEINE" in labelset: korrekt = False
        
        # 3. Gruppenmerkmale - Logik (in beide Richtungen)

        nation = {"Türkischstämmige Deutsche", "Marokkaner:innen", "In Deutschland lebende Ausländer:innen",
                    "Asiat:innen", "Pol:innen", "Afrikaner:innen", "Syrer:innen", "Palästinenser:innen"}
        herkunft = {"POC", "Araber:in"}
        religion = {"Muslim:e/innen", "Juden/Jüdinnen", "Christ:innen"}
        polit = {"Die Grünen", "die SPD", "die Linke", "CDU/CSU", "AfD", "Nazis", "Islamist:innen", "Kommunist:innen",
                    "Zionist:innen", "NPD", "FDP", "FridaysForFuture", "Pegida", "Anarchist:in"}
        geschlecht = {"Trans/NB-Personen", "Frauen", "Männer"}
        andere = {"Flüchtlinge", "Asylbewerber:innen", "Sich illegal in Deutschland aufhaltende Personen", "Migrant:innen", "Vorbestrafte",
                    "Veganer", "Senior:innen", "Lesben, Schwule, Bi", "Kinder", "Jugendliche", "Polizist:innen", "Obdachlose",
                    "Richter:innen", "Analphabet:innen", "Soldat:innen", "Behinderte"}

        # Richtung 1: falls Gruppe vorhanden, korrektes Merkmal auch vorhanden
        for i in nation & labelset:
            if "Nationalität" not in labelset:
                korrekt = False
        for j in herkunft & labelset:
            if 'ethnische Herkunft / "Rasse"' not in labelset:
                korrekt = False
        for k in religion & labelset:
            if "Religion / Weltanschauung" not in labelset:
                korrekt = False
        for l in polit & labelset:
            if "Politische Einstellung" not in labelset:
                korrekt = False
        for m in geschlecht & labelset:
            if "Geschlecht" not in labelset:
                korrekt = False
        for n in andere & labelset:
            if "Anderes Merkmal" not in labelset:
                korrekt = False

        # Richtung 2: falls Merkmal vorhanden, auch eine passende Gruppe vorhanden
        if "Nationalität" in labelset:
            if nation & labelset == set():
                korrekt = False
        if 'ethnische Herkunft / "Rasse"' in labelset:
            if herkunft & labelset == set():
                korrekt = False
        if "Religion / Weltanschauung" in labelset:
            if religion & labelset == set():
                korrekt = False
        if "Politische Einstellung" in labelset:
            if polit & labelset == set():
                korrekt = False
        if "Geschlecht" in labelset:
            if geschlecht & labelset == set():
                korrekt = False
        if "Anderes Merkmal" in labelset:
            if andere & labelset == set():
                korrekt = False

    return korrekt

In [ ]:
# Probleme in der Annotationslogik von Hand durchgehen; jeweils für die Trainingsdaten und die Testdaten

# Beispieleintrag der Annotations-Json-Datei:
# { "id": "01112520",
#   "data": "@SteiblBarbara @Thomas_S_Wagner @RitaKratzert Weitaus schlimmer. ...",
#   "label": ["KEINE"],
#   "tag1": "NEG",
#   "tag2": "HATE"}

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_annotiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

probleme = check_anno(train_annotations)

In [ ]:
# Jeder fehlerhafte Eintrag wurde einzeln von Hand korrigiert
# Hier: Liste der Korrekturen

new01110262 = {'id': '01110262', 'label': ['KEINE', 'Flüchtlinge', 'Afrikaner:innen', 'Nationalität', 'Anderes Merkmal']}
new03113327 = {'id': '03113327', 'label': ['Angriff der Menschenwürde', 'VVH-ALLG', 'POC', 'ethnische Herkunft / "Rasse"', 'Kinder', 'Die Grünen', 'Politische Einstellung', 'Anderes Merkmal'] }
new01114326 = {'id': '01114326', 'label': ['Muslim:e/innen', 'Religion / Weltanschauung', 'Islamist:innen', 'Aufstachelung zu Hass', 'VVH-ALLG', 'Politische Einstellung']}
new01111028 = {'id': '01111028', 'label': ['KEINE', 'Araber:in', 'ethnische Herkunft / "Rasse"']}
new03113648 = {'id': '03113648', 'label': ['KEINE', 'Politische Einstellung', 'Nazis']}
new01112254 = {'id': '01112254', 'label': ['KEINE', 'Pol:innen', 'In Deutschland lebende Ausländer:innen', 'Nationalität']}
new02110710 = {'id': '02110710', 'label': ['KEINE', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'ethnische Herkunft / "Rasse"', 'POC', 'Afrikaner:innen', 'Nationalität']}
new04111621 = {'id': '04111621', 'label': ['KEINE']}
new03112728 = {'id': '03112728', 'label': ['KEINE']}
new02113786 = {'id': '02113786', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new01113048 = {'id': '01113048', 'label': ['Palästinenser:innen', 'Nationalität', 'VVH-ALLG', 'Aufstachelung zu Hass'], 'tag1': 'NEG', 'tag2': 'HATE'}
new01110156 = {'id': '01110156', 'label': ['Palästinenser:innen', 'Nationalität', 'VVH-ALLG', 'Aufstachelung zu Hass'], 'tag1': 'NEG', 'tag2': 'HATE'}
new01222281 = {'id': '01222281', 'label': ['Die Grünen', 'Türkischstämmige Deutsche','Nationalität', 'Politische Einstellung', 'KEINE']}
new02220290 = {'id': '02220290', 'label': ['KEINE', 'die SPD', 'Politische Einstellung']}
new01221146 = {'id': '01221146', 'label': ['KEINE', 'die SPD', 'Politische Einstellung']}
new01220517 = {'id': '01220517', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Pol:innen', 'Migrant:innen', 'Nationalität', 'Anderes Merkmal']}
new02221882 = {'id': '02221882', 'label': ['KEINE', 'Zionist:innen', 'Palästinenser:innen', 'Politische Einstellung', 'Nationalität']}
new01223120 = {'id': '01223120', 'label': ['KEINE']}
new01221025 = {'id': '01221025', 'label': ['Verharmlosen', 'VVH-NS']}
new02222036 = {'id': '02222036', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new01220250 = {'id': '01220250', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new02221762 = {'id': '02221762', 'label': ['KEINE', 'AfD', 'Politische Einstellung', 'Islamist:innen', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'Nationalität', 'ethnische Herkunft / "Rasse"']}
new01223162 = {'id': '01223162', 'label': ['KEINE', 'Islamist:innen', 'Politische Einstellung']}
new02221146 = {'id': '02221146', 'label': ['Nazis', 'Aufforderung zu Gewalt- oder Willkürmaßnahmen', 'VVH-ALLG', 'Politische Einstellung']}
new01220249 = {'id': '01220249', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Nationalität']}
new02221991 = {'id': '02221991', 'label': ['KEINE', 'Palästinenser:innen', 'Zionist:innen', 'Politische Einstellung', 'Nationalität']}
new02220270 = {'id': '02220270', 'label': ['KEINE', 'Araber:in', 'Afrikaner:innen', 'In Deutschland lebende Ausländer:innen', 'Nationalität', 'Religion / Weltanschauung', 'Muslim:e/innen', 'ethnische Herkunft / "Rasse"']}
new02220311 = {'id': '02220311', 'label': ['KEINE', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'Afrikaner:innen', 'Muslim:e/innen', 'Religion / Weltanschauung', 'Nationalität', 'ethnische Herkunft / "Rasse"']}
new01222167 = {'id': '01222167', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Islamist:innen', 'Die Grünen', 'Politische Einstellung', 'Asylbewerber:innen', 'Anderes Merkmal', 'Nationalität']}
new01222700 = {'id': '01222700', 'label': ['KEINE', 'Anderes Merkmal', 'Sich illegal in Deutschland aufhaltende Personen']}
new01222612 = {'id': '01222612', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}

In [ ]:
# Erkannte Fehler: Tweets korrigiert in neue Datei schreiben
# Dabei inkl. der Korrektur eines Off-by-One-Fehlers in den IDs, der auch erst im Laufe der Annotation aufgefallen war (je +1)
# Bereits ausgeführt, nicht noch einmal ausführen

# Korrekturen in den Trainingsdaten (noch wie in den Ursprungsdatensätzen getrennt, s. Anmerkung oben)
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_annotiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

korrekturen_train = [new01110262, new03113327, new01114326, new01111028, new03113648, new01112254, new02110710, new04111621, new03112728, new02113786, new01113048, new01110156]
for i in train_annotations:
    for j in korrekturen_train:
        if i["id"] == j["id"]:
            i["label"] = j["label"]
    i["id"] = i["id"][:4] + f'{int(i["id"][4:])+1:04d}'
train_anno_k = json.dumps(train_annotations)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="w", encoding="utf-8") as trainf_k:
    trainf_k.write(train_anno_k)

# Korrekturen in den Testdaten
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_annotiert.json", mode="r", encoding="utf-8") as testf:
    test_anno = testf.read()
    test_annotations = json.loads(test_anno)

korrekturen_test = [new01222281, new02220290, new01221146, new01220517, new02221882, new01223120, new01221025, new02222036, new01220250,
                    new02221762, new01223162, new02221146, new01220249, new02221991, new02220270, new02220311, new01222167, new01222700, new01222612]
for i in test_annotations:
    for j in korrekturen_test:
        if i["id"] == j["id"]:
            i["label"] = j["label"]
    i["id"] = i["id"][:4] + f'{int(i["id"][4:])+1:04d}'
test_anno_k = json.dumps(test_annotations)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_anno_korrigiert.json", mode="w", encoding="utf-8") as testf_k:
    testf_k.write(test_anno_k)

### 3. Annotationen nach Merkmal getrennt speichern

- in unterschiedlicher Detailgenauigkeit:
    - VVH Ja / Nein
    - Gruppe Ja / Nein, Welches Gruppenmerkmal / Keine Gruppe 
    - Handlung VVH-Allg Ja / Nein, Welche Handlung genau / Keine Handlung

- gleichzeitig @user-Erwähnungen anonymisieren


In [ ]:
import re
import json
import copy


# Liste der zu entfernenden Duplikate laden (schwarze Liste)
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\schwarze_Liste.txt", mode="r", encoding="utf-8") as schw:
    schwarzeListe = schw.readlines()
    schwarzeListe = [line.strip() for line in schwarzeListe]

def anonym_atuser(tweet):
    tweet_anonym = re.sub('@[^@ ]+?@', '@user@', tweet)
    tweet_anonym = re.sub('@[^@ ]+? ', '@user ', tweet_anonym)
    tweet_anonym = re.sub('@[^@ ]+?$', '@user', tweet_anonym)
    return tweet_anonym

# Korrigierte Annotationen laden
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as in_hate_train:
    hate_train = in_hate_train.read()
    hate_train_anno = json.loads(hate_train)
    for entry in hate_train_anno:
        entry["data"] = anonym_atuser(entry["data"])

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as in_hate_test:
    hate_test = in_hate_test.read()
    hate_test_anno = json.loads(hate_test)
    for entry in hate_test_anno:
        entry["data"] = anonym_atuser(entry["data"])

hate_anno = hate_train_anno + hate_test_anno

# Einheitliche Formatierung
for entry in hate_anno:
    entry.pop("tag1")
    entry.pop("tag2")
    entry["corpus_id"] = entry.pop("id")
    entry["tweet"] = entry.pop("data")

# aussortierte IDs löschen
hate_anno = [entry for entry in hate_anno if entry["corpus_id"] not in schwarzeListe]    

In [ ]:
# 1. Annotation Volksverhetzung Ja / Nein ("VVH", "KEINE")

def transform_anno_vvh(corp):
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if "KEINE" in entry["label"]:
            new_entry["label"] = "KEINE"
        else:
            new_entry["label"] = "VVH"
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_VVH.json", mode="w", encoding="utf-8") as out_vvh:
    vvh_keep = transform_anno_vvh(hate_anno)
    vvh = json.dumps(vvh_keep)
    out_vvh.write(vvh)

In [ ]:
# 2.1   Gruppe Ja / Nein ("Gruppe", "KeineGruppe")

def transform_anno_gruppe(corp):
    gruppe = {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if gruppe & set(entry["label"]) != set():
            new_entry["label"] = "Gruppe"
        else:
            new_entry["label"] = "KeineGruppe"
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_Gruppe.json", mode="w", encoding="utf-8") as out_gr:
    gruppe_keep = transform_anno_gruppe(hate_anno)
    gruppe = json.dumps(gruppe_keep)
    out_gr.write(gruppe)


# 2.2   Das die Gruppe definierende Merkmal / Keine Gruppe

def transform_anno_gruppe_det(corp):
    gruppe = {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if gruppe & set(entry["label"]) != set():
            new_entry["label"] = list(gruppe & set(entry["label"]))
        else:
            new_entry["label"] = ["KeineGruppe"]
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_GruppeDetail.json", mode="w", encoding="utf-8") as out_grd:
    gruppedetail = transform_anno_gruppe_det(hate_anno)
    grdt = json.dumps(gruppedetail)
    out_grd.write(grdt)

In [ ]:
# 3.1   Handlung VVH-Allg Ja / Nein

def transform_anno_handlung(corp):
    handlung = {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if handlung & set(entry["label"]) != set():
            new_entry["label"] = "Handlung"
        else:
            new_entry["label"] = "KeineHandlung"
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_Handlung.json", mode="w", encoding="utf-8") as out_handl:
    handlung_keep = transform_anno_handlung(hate_anno)
    hndl = json.dumps(handlung_keep)
    out_handl.write(hndl)


# 3.2   Welche Handlung VVH-Allg / Keine Handlung 

def transform_anno_handlung_det(corp):
    handlung = {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if handlung & set(entry["label"]) != set():
            new_entry["label"] = list(handlung & set(entry["label"]))
        else:
            new_entry["label"] = ["KeineHandlung"]
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_HandlungDetail.json", mode="w", encoding="utf-8") as out_handld:
    handdetail = transform_anno_handlung_det(hate_anno)
    hddt = json.dumps(handdetail)
    out_handld.write(hddt)